In [11]:
pwd

'/home/uniq/Documents/Deep Learning/VANILLA GAN'

In [12]:
import os 

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets
import matplotlib.pyplot as plt

from PIL import Image
from torchsummary import summary


In [13]:
#Hyper parameters
lr = 0.001
batch_size = 64
num_epochs = 10
img_path = './data/Male and Female face dataset'  
device = torch. device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [14]:
# Define transformation for preprocessing
transform = transforms.Compose([
    transforms.Resize((64)),  # Resize images to 128x128
    transforms.CenterCrop(64),  # Crop images to 128x128 about
    transforms.ToTensor(),                       # Convert image to tensor
    transforms.Normalize([0.5], [0.5])           # Normalize to [-1, 1]
])

# Dataset Preparations

In [15]:
class faceDataset():
    def __init__(self, img_path, transform=None):
        self.img_path = img_path
        self.transform = transform
        self.image_paths = []
        
       # Collect all image paths
        for subdir, _, files in os.walk(img_path):
            for file in files:
                if file.endswith(('jpg', 'jpeg', 'png')):  # Filter for image files
                    self.image_paths.append(os.path.join(subdir, file))
            print(f"Found {len(self.image_paths)} images")
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')  # Open image in RGB mode
        
        if self.transform:
            image = self.transform(image)
        return image
    
faceData = faceDataset(img_path, transform=transform)
faceData[0].shape


Found 0 images
Found 2717 images
Found 5400 images


torch.Size([3, 64, 64])

In [16]:
faceDataloader = DataLoader(faceData, batch_size = batch_size, shuffle=True)

In [17]:
next(iter(faceDataloader)).shape

torch.Size([64, 3, 64, 64])

## Model Building

In [18]:
# Define the Generator (U-Net style)
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        # Define a simple U-Net like architecture
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1), nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1), nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1), nn.ReLU(),
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1), nn.Tanh(),
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [19]:
# Define the Discriminator 
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1), nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1), nn.LeakyReLU(0.2),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1), nn.LeakyReLU(0.2),
            nn.Conv2d(256, 1, kernel_size=4, padding=1),
        )
    
    def forward(self, x):
        return self.model(x)